In [41]:
import pandas as pd
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_selection import mutual_info_classif
from sklearn.metrics import accuracy_score, classification_report

from itertools import product
from sklearn.metrics import accuracy_score, classification_report

In [42]:
train = pd.read_csv('train_new.csv')
validation = pd.read_csv('val_new.csv')
test = pd.read_csv("test_new.csv")

In [43]:
def do_BOW(df, bow_voc_size):
  df_text = df[['tasks_open', 'verify_open', 'suboptimal_open']].fillna('').agg(' '.join, axis=1)

  vectorizer = CountVectorizer(max_features=bow_voc_size)
  bow = vectorizer.fit_transform(df_text)

  df_numeric = df.drop(columns=['tasks_open', 'verify_open', 'suboptimal_open'])
  df_bow = pd.DataFrame(bow.toarray(), columns=vectorizer.get_feature_names_out(), index=df.index)
  df_final = pd.concat([df_numeric, df_bow], axis=1)

  return df_final, vectorizer

In [44]:
def feature_selection(X, y, k):
  mi_scores = mutual_info_classif(X, y, discrete_features=True)
  feature_scores = pd.Series(mi_scores, index=X.columns)
  selected_features = feature_scores.nlargest(k).index.tolist()
  return selected_features

In [45]:
def train_model(X_train, y_train, bow_voc_size: int, hidden_layer_sizes: tuple, l2_reg_strength: float, learning_rate: float, num_features: int):
    """
    Trains a neural network classifier using all features except open response variables

    Parameters:
    X_train: Training features
    y_train: Training labels
    bow_voc_size: Size of the bag-of-words vocabulary
    hidden_layer_sizes: Number of neurons in each hidden layer
    l2_reg_strength: L2 regularization strength
    learning_rate: Learning rate

    Returns: trained model
    """

    # Make copy to avoid modifying original data
    X_train = X_train.copy()

    # Ignore open responses for now
    # columns_to_drop = ['tasks_open', 'verify_open', 'suboptimal_open']
    # X_train = X_train.drop(columns=columns_to_drop)

    # Bag-of-words
    X_train, vectorizer = do_BOW(X_train, bow_voc_size)

    # Feature selection
    selected_features = feature_selection(X_train, y_train, num_features)
    X_train = X_train[selected_features]

    # Create and train the neural network
    print("Training neural network...")
    model = MLPClassifier(
        hidden_layer_sizes=hidden_layer_sizes,
        activation='relu',
        solver='sgd',
        alpha=l2_reg_strength,  # L2 regularization
        batch_size=60,
        learning_rate_init=learning_rate,
        max_iter=400,
        n_iter_no_change=5 # maximum number of epochs
    )

    # Train the model
    model.fit(X_train, y_train)

    # Training evaluation
    train_accuracy = model.score(X_train, y_train)
    print(f"\nTraining completed!")
    print(f"Training accuracy: {train_accuracy:.4f}")

    return model, vectorizer, selected_features

In [46]:
def hyperparameter_tuning(X_train, y_train, validation, max_combinations=1000):
    """
    Performs exhaustive hyperparameter tuning with a large parameter space.
    Uses random sampling if combinations exceed max_combinations.

    Parameters:
    X_train: Training features
    y_train: Training labels
    X_val: Validation features
    y_val: Validation labels
    bow_voc_size: Size of the bag-of-words vocabulary
    max_combinations: Maximum number of combinations to try

    Returns: The best accuracy and the corresponding hyperparameters
    """

    # Extensive hyperparameter grid
    bow_voc_size_options = [50, 100, 200, 500, 1000, 5000]

    hidden_layer_options = [
        # Single hidden layer architectures
        # (4,), (8,), (16,), (32,), (64,), (128,),
        # Two hidden layers
        (4, 4), (8, 8), (16, 16), (32, 32), (64, 64), (128, 128),
        (128, 64), (64, 32), (32, 16),
        # Three hidden layers
        (4, 4, 4), (8, 8, 8), (16, 16, 16), (32, 32, 32), (64, 64, 64),
        (128, 128, 128), (128, 64, 32), (64, 32, 16), (32, 16, 8)
    ]

    l2_options = [0.0001, 0.0005, 0.001, 0.005, 0.01]

    learning_rate_options = [0.0001, 0.0005, 0.001, 0.005, 0.01]

    num_features = [10, 25, 50, 100]

    # Create all possible combinations
    all_combinations = list(product(bow_voc_size_options, hidden_layer_options, l2_options, learning_rate_options, num_features))

    print(f"Total possible combinations: {len(all_combinations)}")

    # Sample combinations if there are too many
    if len(all_combinations) > max_combinations:
        selected_indices = np.random.choice(
            len(all_combinations),
            size=max_combinations,
            replace=False
        )
        combinations_to_try = [all_combinations[i] for i in selected_indices]
        print(f"Sampling {max_combinations} random combinations for efficiency")
    else:
        combinations_to_try = all_combinations

    best_accuracy = 0
    best_params = {}
    best_model = None
    best_vectorizer = None
    best_selected_features = None

    for i, (bow_voc_size, hidden_layer_sizes, l2_reg_strength, learning_rate, num_features) in enumerate(combinations_to_try):
        try:
            # You'll need to modify your train_model function to accept these additional parameters
            model, vectorizer, selected_features = train_model(X_train, y_train, bow_voc_size,
                                        hidden_layer_sizes, l2_reg_strength,
                                        learning_rate, num_features)

            X_val, y_val = get_X_val_y_val(validation, vectorizer)
            X_val = X_val[selected_features]

            # Calculate validation accuracy
            val_accuracy = model.score(X_val, y_val)

            # Update best combination if improved
            if val_accuracy > best_accuracy:
                best_accuracy = val_accuracy
                best_params = {
                    'bow_voc_size': bow_voc_size,
                    'hidden_layer_sizes': hidden_layer_sizes,
                    'l2_reg_strength': l2_reg_strength,
                    'learning_rate': learning_rate,
                    'num_features': num_features
                }
                best_model = model
                best_vectorizer = vectorizer
                best_selected_features = selected_features
                print(f"New best! Accuracy: {best_accuracy:.4f} | Voc_size: {bow_voc_size} | Layers: {hidden_layer_sizes} | L2: {l2_reg_strength} | LR: {learning_rate} | Num_features: {num_features}")

            if (i + 1) % 10 == 0:
                print(f"Completed {i+1}/{len(combinations_to_try)} combinations...")

        except Exception as e:
            print(f"Error with combination {i}: {e}")
            continue

    return best_accuracy, best_params, best_model, best_vectorizer, best_selected_features

In [47]:
def get_X_val_y_val(validation, vectorizer):
  X_val = validation.drop(columns=["label"])
  y_val = validation['label']
  X_val_text = X_val[['tasks_open', 'verify_open', 'suboptimal_open']].fillna('').agg(' '.join, axis=1)
  X_val_bow = vectorizer.transform(X_val_text)
  X_val_numeric = X_val.drop(columns=['tasks_open', 'verify_open', 'suboptimal_open'])
  X_val_bow_df = pd.DataFrame(X_val_bow.toarray(), columns=vectorizer.get_feature_names_out(), index=X_val.index)
  X_val= pd.concat([X_val_numeric, X_val_bow_df], axis=1)

  return X_val, y_val

In [ ]:
cur_train = 0
cur_val = 0
cur_model = None
cur_vectorizer = None
cur_selected_features = None

for _ in range(100):
  model, vectorizer, features = train_model(train.drop(columns=["label"]), train['label'], 200, (128, 64, 32), 0.0001, 0.0001, 50)
  X_val, y_val = get_X_val_y_val(validation, vectorizer)
  X_val = X_val[features]
  val_acc = model.score(X_val, y_val)

  if val_acc > cur_val:
    cur_val = val_acc
    cur_model = model
    cur_vectorizer = vectorizer
    cur_selected_features = features
    print("New Best!!!")

Training neural network...


/usr/local/lib/python3.12/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (400) reached and the optimization hasn't converged yet.
  warnings.warn(



Training completed!
Training accuracy: 0.7153
New Best!!!
Training neural network...


/usr/local/lib/python3.12/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (400) reached and the optimization hasn't converged yet.
  warnings.warn(



Training completed!
Training accuracy: 0.6962
New Best!!!
Training neural network...


...


Training completed!
Training accuracy: 0.7135
Training neural network...


/usr/local/lib/python3.12/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (400) reached and the optimization hasn't converged yet.
  warnings.warn(



Training completed!
Training accuracy: 0.7118
Training neural network...


/usr/local/lib/python3.12/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (400) reached and the optimization hasn't converged yet.
  warnings.warn(



Training completed!
Training accuracy: 0.7031
Training neural network...


In [34]:
cur_val

0.7317073170731707

In [35]:
cur_vectorizer.get_feature_names_out()

array(['also', 'and', 'answer', 'are', 'as', 'ask', 'asked', 'but', 'by',
       'can', 'check', 'code', 'coding', 'for', 'google', 'have', 'if',
       'in', 'is', 'it', 'its', 'like', 'math', 'me', 'model', 'my',
       'not', 'of', 'on', 'or', 'questions', 'response', 'search', 'some',
       'suboptimal', 'tasks', 'that', 'the', 'this', 'to', 'up', 'use',
       'used', 'usually', 'verify', 'was', 'when', 'with', 'would',
       'writing'], dtype=object)

In [36]:
cur_selected_features

['academic_scale',
 'Explaining complex concepts simply',
 'suboptimal_scale',
 'Drafting professional text ',
 'to',
 'Writing or debugging code',
 'verify_scale',
 'the',
 'Converting content between formats ',
 'it']

In [40]:
cur_model.classes_

array(['ChatGPT', 'Claude', 'Gemini'], dtype='<U7')

In [37]:
X_test, y_test = get_X_val_y_val(test, vectorizer)
X_test = X_test[features]
model.score(X_test,y_test)

0.6666666666666666

In [39]:
params = {'weights': cur_model.coefs_,'intercepts': cur_model.intercepts_}
np.save("mlp_params", params, allow_pickle=True)

In [48]:
best_accuracy, best_params, best_model, best_vectorizer, best_selected_features = hyperparameter_tuning(train.drop(columns=["label"]), train['label'], validation)

Total possible combinations: 10800
Sampling 1000 random combinations for efficiency
Training neural network...


/usr/local/lib/python3.12/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (400) reached and the optimization hasn't converged yet.
  warnings.warn(



Training completed!
Training accuracy: 0.6806
New best! Accuracy: 0.6260 | Voc_size: 200 | Layers: (64, 32) | L2: 0.01 | LR: 0.0001 | Num_features: 50
Training neural network...

Training completed!
Training accuracy: 0.9809
Training neural network...


/usr/local/lib/python3.12/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (400) reached and the optimization hasn't converged yet.
  warnings.warn(



Training completed!
Training accuracy: 0.6528
Training neural network...


...

/usr/local/lib/python3.12/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (400) reached and the optimization hasn't converged yet.
  warnings.warn(



Training completed!
Training accuracy: 0.6667
Training neural network...


/usr/local/lib/python3.12/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (400) reached and the optimization hasn't converged yet.
  warnings.warn(



Training completed!
Training accuracy: 0.5538
Training neural network...

Training completed!
Training accuracy: 0.7917
Training neural network...

Training completed!
Training accuracy: 0.7396
Completed 1000/1000 combinations...


/usr/local/lib/python3.12/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (400) reached and the optimization hasn't converged yet.
  warnings.warn(


In [49]:
best_accuracy

0.7479674796747967

In [50]:
best_params

{'bow_voc_size': 200,
 'hidden_layer_sizes': (128, 64, 32),
 'l2_reg_strength': 0.0001,
 'learning_rate': 0.0001,
 'num_features': 50}

In [51]:
best_model

MLPClassifier(batch_size=60, hidden_layer_sizes=(128, 64, 32),
              learning_rate_init=0.0001, max_iter=400, n_iter_no_change=5,
              solver='sgd')

In [52]:
best_vectorizer.get_feature_names_out()

array(['about', 'ai', 'all', 'also', 'am', 'an', 'and', 'another',
       'answer', 'answers', 'any', 'are', 'as', 'ask', 'asked', 'asking',
       'at', 'based', 'be', 'because', 'before', 'better',
       'brainstorming', 'but', 'by', 'can', 'check', 'checking', 'code',
       'coding', 'compare', 'complex', 'computations', 'concept',
       'concepts', 'content', 'correct', 'creative', 'cross', 'data',
       'debug', 'debugging', 'did', 'didn', 'different', 'do', 'does',
       'doesn', 'don', 'emails', 'errors', 'essay', 'even', 'example',
       'explain', 'explaining', 'explanation', 'find', 'follow', 'for',
       'found', 'from', 'gave', 'general', 'generate', 'generating',
       'give', 'gives', 'go', 'good', 'google', 'had', 'has', 'have',
       'help', 'how', 'ideas', 'if', 'in', 'incorrect', 'information',
       'into', 'is', 'it', 'its', 'just', 'like', 'logic', 'long', 'look',
       'made', 'make', 'makes', 'math', 'mathematical', 'me', 'model',
       'models', 'mor

In [53]:
best_selected_features

['academic_scale',
 'Explaining complex concepts simply',
 'Drafting professional text ',
 'suboptimal_scale',
 'to',
 'Converting content between formats ',
 'verify_scale',
 'Writing or debugging code',
 'the',
 'concepts',
 'it',
 'and',
 'sub_Math computations',
 'of',
 'another',
 'model',
 'ask',
 'was',
 'Writing or editing essays/reports',
 'for',
 'google',
 'never',
 'coding',
 'sub_Writing or debugging code',
 'wrong',
 'something',
 'ref_scale',
 'that',
 'usually',
 'answer',
 'my',
 'concept',
 'code',
 'math',
 'if',
 'or',
 'me',
 'search',
 'used',
 'as',
 'which',
 'on',
 'in',
 'gave',
 'questions',
 'about',
 'question',
 'emails',
 'Brainstorming or generating creative ideas',
 'proof']

In [54]:
X_test, y_test = get_X_val_y_val(test, best_vectorizer)
X_test = X_test[best_selected_features]

In [55]:
best_model.score(X_test, y_test)

0.6111111111111112